# 07_04: Cleaning and tidying data

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
df = pd.read_html('Athletics_at_the_2024_Summer_Olympics.html', match='Lyles')[0]
df.head(10)

In [ ]:
df = df.dropna(how='all')
df.head(10)

In [ ]:
df.Event = df.Event.str.slice(0, -8)
df.head()

In [ ]:
athletes = df.melt(id_vars=['Event'], value_vars=['Gold', 'Silver', 'Bronze'],
                   var_name='Medal', value_name='Athlete')
athletes

In [ ]:
results = df.melt(id_vars=['Event'], value_vars=['Gold.1', 'Silver.1', 'Bronze.1'],
                  var_name='Medal', value_name='Result')
results

In [ ]:
results.Medal = results.Medal.str.replace('.1', '')
results

In [ ]:
medals = pd.merge(athletes, results, on=['Event', 'Medal'])
medals

In [ ]:
medals.Athlete[0]

In [ ]:
medals['Country'] = medals.Athlete.str.split(' \xa0').str.get(1)
medals['Athlete'] = medals.Athlete.str.split(' \xa0').str.get(0)

In [ ]:
medals.head()

In [ ]:
relay_events = medals.Event.str.contains('relay') # identify the relay events
medals[relay_events]

In [ ]:
def whichcountry(s):
    words = s.split(' ')
    
    if s.startswith('Canada') or s.startswith('Botswana'):
        return words[0]
    else:
        return words[0] + ' ' + words[1]

In [ ]:
medals.loc[relay_events].Athlete.apply(whichcountry)

In [ ]:
medals.loc[relay_events, 'Country'] = medals.loc[relay_events].Athlete.apply(whichcountry)

In [ ]:
medals.loc[relay_events]

In [ ]:
medals.Result.unique()

In [ ]:
medals.Result = medals.Result.str.replace(r' \(.*\)', '', regex=True) \
                             .str.replace(r' (pts|m)', '', regex=True)

In [ ]:
medals.head()

In [ ]:
medals['Record'] = medals.Result.str.split(' ').str.slice(1,).str.join(' ')

medals.Result = medals.Result.str.split(' ').str.get(0)

In [ ]:
medals.Record.unique()

In [ ]:
medals

In [ ]:
medals[medals.Record.str.contains('OR')]

In [ ]:
medals[(medals.Medal == 'Bronze') & (medals.Record.str.contains('PB') | medals.Record.str.contains('SB'))]

In [ ]:
medals.value_counts('Athlete')